In [3]:
import torch
from IPython.display import Audio
from pprint import pprint
import soundfile         as sf

In [2]:
SAMPLING_RATE = 16000
torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

  0%|          | 0.00/1.83M [00:00<?, ?B/s]

In [6]:
def get_wav_duration(wav_file):
    f                   = sf.SoundFile(wav_file)
    duration_in_seconds = f.frames / f.samplerate    
    return duration_in_seconds

In [7]:
get_wav_duration("./en_example.wav")

60.0

In [10]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=False,
                              )

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

Using cache found in /home/amitli/.cache/torch/hub/snakers4_silero-vad_master


In [11]:
wav = read_audio('en_example.wav', sampling_rate=SAMPLING_RATE)

In [16]:
len(wav), 60*16000, 960000/1536, 1536*1000/16000

(960000, 960000, 625.0, 96.0)

In [19]:
vad_iterator = VADIterator(model)

https://colab.research.google.com/github/snakers4/silero-vad/blob/master/silero-vad.ipynb#scrollTo=BX3UgwwB2Fwv

In [37]:
speech_probs = []
window_size_samples = 16000 #1536
for i in range(0, len(wav), window_size_samples):
    chunk = wav[i: i+ window_size_samples]
    if len(chunk) < window_size_samples:
        break
        
    speech_dict = vad_iterator(chunk, return_seconds=True)
    if speech_dict:
        print(speech_dict, end=' ')
        print("")
    
    speech_prob = model(chunk, SAMPLING_RATE).item()
    speech_probs.append(speech_prob)
vad_iterator.reset_states() # reset model states after each audio

print("\n\n")
print(speech_probs[:10]) # first 10 chunks predicts

{'start': 1.0} 
{'end': 8.0} 
{'start': 10.0} 
{'end': 48.0} 
{'start': 50.0} 



[0.9595266580581665, 0.938676655292511, 0.3671853542327881, 0.9120056629180908, 0.4447315037250519, 0.9862000942230225, 0.6391571760177612, 0.023880869150161743, 0.01113173272460699, 0.7861336469650269]


In [35]:
len(speech_probs)

60

In [24]:
625*1536

960000